In [1]:
setwd("~/Projects/livemanager")
load("plm.RData")

In [2]:
odds <- read.csv("data/match_odds.csv")
next_odds <- read.csv("data/next_match_odds.csv")
str(odds)

'data.frame':	234 obs. of  18 variables:
 $ LEAGUE   : Factor w/ 1 level "Bundesliga": 1 1 1 1 1 1 1 1 1 1 ...
 $ FTR      : Factor w/ 3 levels "A","D","H": 3 3 3 3 2 1 3 2 3 3 ...
 $ FTSC     : Factor w/ 28 levels "0:0","0:1","0:2",..: 12 6 25 17 14 3 6 20 12 12 ...
 $ MATCH_URL: Factor w/ 234 levels "http://www.oddsportal.com/soccer/germany/bundesliga/augsburg-b-monchengladbach-GncNw7cb/",..: 77 33 52 18 54 114 144 155 128 100 ...
 $ SEASON   : int  2015 2015 2015 2015 2015 2015 2015 2015 2015 2015 ...
 $ HOMETEAM : Factor w/ 18 levels "1. FC Koln","Augsburg",..: 7 4 5 3 6 10 12 13 11 9 ...
 $ TIME     : Factor w/ 10 levels "13:30","14:30",..: 5 2 6 2 2 2 2 2 10 5 ...
 $ FTHG     : int  2 1 5 3 2 0 1 3 2 2 ...
 $ AVOD     : num  5.14 3.97 11 4.55 3.18 3.31 3.7 3.31 3.24 3.15 ...
 $ AWAYTEAM : Factor w/ 18 levels "1. FC Koln","Augsburg",..: 14 9 17 8 2 1 18 16 15 11 ...
 $ AVOA     : num  8.85 5.44 24.43 6.21 2.75 ...
 $ COUNTRY  : Factor w/ 1 level "Germany": 1 1 1 1 1 1 1 1 1 1 ...


In [3]:
## Express Odds as Chances
next_odds$Hprob <- (1 / next_odds$AVOH) / ((1/next_odds$AVOH) + (1/next_odds$AVOD) + (1/next_odds$AVOA))
next_odds$Aprob <- (1 / next_odds$AVOA) / ((1/next_odds$AVOH) + (1/next_odds$AVOD) + (1/next_odds$AVOA))
odds$Hprob <- (1 / odds$AVOH) / ((1/odds$AVOH) + (1/odds$AVOD) + (1/odds$AVOA))
odds$Aprob <- (1 / odds$AVOA) / ((1/odds$AVOH) + (1/odds$AVOD) + (1/odds$AVOA))


In [4]:
#next_odds
#head(odds)
levels(odds$HOMETEAM)

[1] "1. FC Koln"          "Augsburg"            "B. Monchengladbach" 
 [4] "Bayer Leverkusen"    "Bayern Munich"       "Darmstadt"          
 [7] "Dortmund"            "Eintracht Frankfurt" "Hamburger SV"       
[10] "Hannover"            "Hertha Berlin"       "Hoffenheim"         
[13] "Ingolstadt"          "Mainz"               "Schalke"            
[16] "VfB Stuttgart"       "Werder Bremen"       "Wolfsburg"

In [5]:
## Align Teamnames with PLM format
levels(next_odds$HOMETEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(next_odds$AWAYTEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(odds$HOMETEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")
levels(odds$AWAYTEAM) <- c("KOE", "FCA", "BMG", "B04", "FCB", "D98", "BVB", "SGE", "HSV", "H96", 
                           "BSC", "TSG", "FCI", "M05", "S04", "VFB", "BRE", "WOB")

## Add matchday column
matchdays <- nrow(odds) / 9
odds$matchday <- rep(matchdays:1, each = 9)

In [6]:
sqldf("SELECT timestamp, DATE, COUNT(1) FROM odds GROUP BY timestamp, DATE ORDER BY timestamp")
#odds[80:120,]
#rep(matchdays:1, each = 9)
#odds$matchday

ERROR: Error in eval(expr, envir, enclos): could not find function "sqldf"


In [7]:
levels(plm$home_shortname) <- factor(plm$Club)
levels(plm$away_shortname) <- factor(plm$Club)

In [8]:
## Get average earnings per team / match
library("dplyr")
plm.clubmatches <- plm %>%
    filter(!is.na(Club) & time_on_pitch > 0) %>%
    group_by(matchday, Club) %>%
    summarise(home = ceiling(mean(Club == home_shortname)),
              earnings = mean(total_earnings)) %>%
    ungroup() %>%
    arrange(matchday, Club)

head(plm.clubmatches, 20)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



,matchday,Club,home,earnings
1,1,B04,0,129208.3
2,1,BMG,0,-692.3077
3,1,BRE,0,9321.429
4,1,BSC,NA,109846.2
5,1,BVB,0,186961.5
6,1,D98,0,62642.86
7,1,FCA,NA,19607.14
8,1,FCB,NA,203142.9
9,1,FCI,NA,109285.7
10,1,H96,0,66928.57


In [9]:
#probs.home
subset(odds, odds$matchday == 9)

,LEAGUE,FTR,FTSC,MATCH_URL,SEASON,HOMETEAM,TIME,FTHG,AVOD,AWAYTEAM,AVOA,COUNTRY,DATE,ODDS_TYPE,FTAG,AVOH,TIMESTAMP,PAGE_URL,Hprob,Aprob,matchday
154,Bundesliga,H,1:0,http://www.oddsportal.com/soccer/germany/bundesliga/vfb-stuttgart-ingolstadt-r7h8d3jB/,2015,VFB,15:30,1,3.76,FCI,4.21,Germany,18-10-2015,EU Odds,0,1.85,1445182200,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.5177454,0.2275128,9
155,Bundesliga,A,0:1,http://www.oddsportal.com/soccer/germany/bundesliga/koln-hannover-z5XnkLro/,2015,KOE,13:30,0,3.65,H96,4.46,Germany,18-10-2015,EU Odds,1,1.83,1445175000,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.5230991,0.2146348,9
156,Bundesliga,A,1:5,http://www.oddsportal.com/soccer/germany/bundesliga/eintracht-frankfurt-b-monchengladbach-YeoLg1zU/,2015,SGE,16:30,1,3.49,BMG,2.7,Germany,17-10-2015,EU Odds,5,2.59,1445099400,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.3701812,0.3550998,9
157,Bundesliga,A,0:2,http://www.oddsportal.com/soccer/germany/bundesliga/augsburg-darmstadt-xUiCeN5H/,2015,FCA,13:30,0,3.53,D98,4.44,Germany,17-10-2015,EU Odds,2,1.87,1445088600,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.5125797,0.2158838,9
158,Bundesliga,D,0:0,http://www.oddsportal.com/soccer/germany/bundesliga/hamburger-bayer-leverkusen-U3l4cqy5/,2015,HSV,13:30,0,3.6,B04,1.87,Germany,17-10-2015,EU Odds,0,4.34,1445088600,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.2209256,0.5127364,9
159,Bundesliga,H,2:1,http://www.oddsportal.com/soccer/germany/bundesliga/schalke-hertha-berlin-8l0ea56h/,2015,S04,13:30,2,3.58,BSC,3.97,Germany,17-10-2015,EU Odds,1,1.96,1445088600,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.4899106,0.2418702,9
160,Bundesliga,A,0:1,http://www.oddsportal.com/soccer/germany/bundesliga/werder-bremen-bayern-munich-GKjGfsLN/,2015,BRE,13:30,0,7.52,FCB,1.19,Germany,17-10-2015,EU Odds,1,14.94,1445088600,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.06434458,0.8078219,9
161,Bundesliga,H,4:2,http://www.oddsportal.com/soccer/germany/bundesliga/wolfsburg-hoffenheim-nuaabPLb/,2015,WOB,13:30,4,4.3,TSG,5.38,Germany,17-10-2015,EU Odds,2,1.61,1445088600,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.5974875,0.178802,9
162,Bundesliga,A,0:2,http://www.oddsportal.com/soccer/germany/bundesliga/mainz-dortmund-l8LUD4Lo/,2015,M05,18:30,0,4.7,BVB,1.49,Germany,16-10-2015,EU Odds,2,6.29,1445020200,http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/,0.1524443,0.6435399,9


In [10]:
#odds['matchday']

probs.home <- odds[c('matchday', 'HOMETEAM', 'Hprob')]
probs.away <- odds[c('matchday', 'AWAYTEAM', 'Aprob')]
names <- c("matchday", "Club", "Prob")
colnames(probs.home) <- names
colnames(probs.away) <- names
probs <- rbind(probs.home, probs.away)
subset(probs, probs$matchday == 9)

,matchday,Club,Prob
154,9,VFB,0.5177454
155,9,KOE,0.5230991
156,9,SGE,0.3701812
157,9,FCA,0.5125797
158,9,HSV,0.2209256
159,9,S04,0.4899106
160,9,BRE,0.06434458
161,9,WOB,0.5974875
162,9,M05,0.1524443
388,9,FCI,0.2275128


In [11]:
subset(plm.clubmatches, plm.clubmatches$matchday == 19)

,matchday,Club,home,earnings
1,19,B04,0,155714.3
2,19,BMG,NA,40272.73
3,19,BRE,0,87272.73
4,19,BSC,0,78461.54
5,19,BVB,NA,144625
6,19,D98,0,7192.308
7,19,FCA,NA,75142.86
8,19,FCB,NA,138071.4
9,19,FCI,0,10076.92
10,19,H96,0,-2888.889


In [12]:
library("sqldf")

Loading required package: gsubfn
Loading required package: proto
Warning message:
In doTryCatch(return(expr), name, parentenv, handler): unable to load shared object '/home/pi/anaconda2/lib64/R/modules//R_X11.so':
  libSM.so.6: cannot open shared object file: No such file or directoryCould not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite
Loading required package: DBI


In [13]:
clubmatches <- plm.clubmatches

earnings_probs <- sqldf("SELECT t1.matchday AS matchday, t1.Club AS Club, home, earnings, Prob
                         FROM clubmatches t1 JOIN probs t2
                         ON t1.matchday = t2.matchday AND t1.Club = t2.Club")
#earnings_probs <- merge(plm.clubmatches, probs, by = c("matchday", "Club"))
subset(earnings_probs, earnings_probs$matchday == 19)

,matchday,Club,home,earnings,Prob
325,19,B04,0,155714.3,0.7512845
326,19,BMG,NA,40272.73,0.3321818
327,19,BRE,0,87272.73,0.3810197
328,19,BSC,0,78461.54,0.3309202
329,19,BVB,NA,144625,0.7535926
330,19,D98,0,7192.308,0.3015563
331,19,FCA,NA,75142.86,0.485737
332,19,FCB,NA,138071.4,0.8798644
333,19,FCI,0,10076.92,0.07835893
334,19,H96,0,-2888.889,0.0905503


In [14]:
library("ggplot2")
ggplot(data = earnings_probs, aes(x = Prob, y = earnings)) +
geom_point() +
geom_smooth()

ERROR: Error in file(con, "rb"): cannot open the connection


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

In [15]:
## Add prob at individual player level
plm.merge <- merge(plm, earnings_probs, by = c("matchday", "Club"), all.x = TRUE)

In [16]:
#options(jupyter.plot_mimetypes = 'image/png')

ggplot(data = subset(plm.merge, plm.merge$time_on_pitch > 0), aes(x=Prob, y = total_earnings)) +
geom_point() +
geom_smooth()

Warning message:
: Removed 143 rows containing non-finite values (stat_smooth).Warning message:
: Removed 143 rows containing missing values (geom_point).

ERROR: Error in file(con, "rb"): cannot open the connection


ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

## Regression model to predict earnings

In [17]:
load("starter_probs.RData")
plm.merge <- merge(plm.merge, starter_probs, by = c("id", "matchday") )

In [18]:
colnames(plm.merge)[ncol(plm.merge)] <- c("probStarter")
subset(plm.merge, plm.merge$id == 64467)

,id,matchday,Club,assist,clean_sheet,goal,game_play_duration,total_earnings,away_shortname,away_squad,away_score,time_on_pitch,status,match_id,period,finished,home_shortname,home_squad,home_score,in_for_player,sub_in_timestamp,sub_in_minutes,sub_out_minutes,sub_out_timestamp,shot_on_target,attempt_saved,successful_pass,Name,init_Value,Pos,home,earnings,Prob,0,probStarter
1,64467,10,KOE,NA,0,NA,90,35000,NA,1562,0,90,starter,8851,FullTime,True,NA,1569,4,NA,1445693400,0,NA,NA,NA,9,25,T. Horn,8.5,GOA,NA,-12428.57,0.04887098,0,1
2,64467,11,KOE,NA,1,NA,90,128500,NA,1577,0,90,starter,8838,FullTime,True,NA,1562,0,NA,1446301800,0,NA,NA,NA,4,35,T. Horn,8.5,GOA,NA,86923.08,0.4388674,0.003996004,0.996004
3,64467,12,KOE,NA,0,NA,90,82500,NA,1562,2,90,starter,8866,FullTime,True,KOE,1564,1,NA,1446906600,0,NA,NA,NA,3,23,T. Horn,8.5,GOA,1,94153.85,0.1563186,0,1
4,64467,13,KOE,NA,1,NA,90,116500,NA,1563,0,90,starter,8856,FullTime,True,NA,1562,0,NA,1448116200,0,NA,NA,NA,1,26,T. Horn,8.5,GOA,NA,80500,0.445153,0,1
5,64467,14,KOE,NA,1,NA,90,126000,NA,1562,0,90,starter,8889,FullTime,True,KOE,1575,0,NA,1448652600,0,NA,NA,NA,1,23,T. Horn,8.5,GOA,1,78142.86,0.3889625,0,1
6,64467,15,KOE,NA,0,NA,90,57000,NA,1568,1,90,starter,8874,FullTime,True,NA,1562,0,NA,1449325800,0,NA,NA,NA,7,31,T. Horn,8.5,GOA,NA,23964.29,0.4198473,0,1
7,64467,16,KOE,NA,0,NA,90,131000,NA,1562,1,90,starter,8907,FullTime,True,KOE,1576,1,NA,1449930600,0,NA,NA,NA,5,23,T. Horn,8.5,GOA,1,67076.92,0.3525251,0,1
8,64467,17,KOE,NA,0,NA,90,66000,KOE,1565,1,90,starter,8892,FullTime,True,NA,1562,2,NA,1450535400,0,NA,NA,NA,2,15,T. Horn,8.5,GOA,NA,87115.38,0.1591019,0,1
9,64467,18,KOE,NA,0,NA,90,28500,NA,1578,3,90,starter,8919,FullTime,True,NA,1562,1,NA,1453559400,0,NA,NA,NA,5,29,T. Horn,8.5,GOA,NA,32769.23,0.4584521,0,1
10,64467,19,KOE,NA,0,NA,90,49000,NA,1562,1,90,starter,8918,FullTime,True,NA,1579,1,NA,1454250600,0,NA,NA,NA,4,16,T. Horn,8.5,GOA,NA,61500,0.1725836,0,1


In [19]:
fit <- lm(total_earnings ~ Prob + init_Value + Pos + probStarter, data = plm.merge)
summary(fit)

## Double-check merges are correct
## R-Squared of 0.29; add running average of earnings (needs to be previous weeks' to serve as predictor).


Call:
lm(formula = total_earnings ~ Prob + init_Value + Pos + probStarter, 
    data = plm.merge)

Residuals:
    Min      1Q  Median      3Q     Max 
-192662  -31459  -10327   18193  505359 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -22343.7     3091.4  -7.228 5.47e-13 ***
Prob         80294.0     4149.2  19.352  < 2e-16 ***
init_Value    2177.6      409.8   5.313 1.11e-07 ***
PosDEF      -20081.2     2041.4  -9.837  < 2e-16 ***
PosGOA      -24083.2     3214.1  -7.493 7.62e-14 ***
PosMID       -5616.8     1945.2  -2.888   0.0039 ** 
probStarter  85669.4     2011.2  42.595  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 56050 on 6581 degrees of freedom
  (4149 observations deleted due to missingness)
Multiple R-squared:  0.2965,	Adjusted R-squared:  0.2958 
F-statistic: 462.2 on 6 and 6581 DF,  p-value: < 2.2e-16


In [20]:
### WHY THE FUCK ARE THERE NO MATCHES FOR BREMEN FOR DAYS 4 & 9???

#sqldf("SELECT Club, matchday FROM plm GROUP BY matchday, Club ORDER BY Club, matchday")
#sqldf("SELECT Club, matchday FROM clubmatches GROUP BY matchday, Club ORDER BY Club, matchday")

BRE49probs <- sqldf("SELECT Club, matchday FROM probs WHERE Club = 'BRE' AND matchday IN (4, 9)
                    GROUP BY matchday, Club ORDER BY Club, matchday")
BRE49cm <- sqldf("SELECT Club, matchday FROM clubmatches WHERE Club = 'BRE' AND matchday IN (4, 9)
                    GROUP BY matchday, Club ORDER BY Club, matchday")